# Signed Networks

This notebook demonstrates analysis of signed networks with positive and negative edges.

## Learning Objectives
- Understand signed network structure
- Analyze balance theory
- Study signed centrality measures
- Implement community detection in signed networks

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Creating Signed Networks

Build and analyze networks with positive and negative edges.

In [ ]:
# Create a signed network
G_signed = nx.Graph()

# Add nodes
nodes = list(range(1, 7))
G_signed.add_nodes_from(nodes)

# Add edges with signs
positive_edges = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6)]
negative_edges = [(1, 3), (2, 4), (3, 5), (4, 6)]

# Add positive edges
for edge in positive_edges:
    G_signed.add_edge(edge[0], edge[1], sign=1)

# Add negative edges
for edge in negative_edges:
    G_signed.add_edge(edge[0], edge[1], sign=-1)

print(f"Signed network: {G_signed.number_of_nodes()} nodes, {G_signed.number_of_edges()} edges")
print(f"Positive edges: {len(positive_edges)}")
print(f"Negative edges: {len(negative_edges)}")

In [ ]:
# Visualize signed network
pos = nx.spring_layout(G_signed, seed=42)

# Separate positive and negative edges
positive_edges = [(u, v) for (u, v, d) in G_signed.edges(data=True) if d['sign'] == 1]
negative_edges = [(u, v) for (u, v, d) in G_signed.edges(data=True) if d['sign'] == -1]

plt.figure(figsize=(10, 8))

# Draw nodes
nx.draw_networkx_nodes(G_signed, pos, node_color='lightblue', node_size=500)

# Draw positive edges (solid lines)
nx.draw_networkx_edges(G_signed, pos, edgelist=positive_edges, 
                        edge_color='green', width=2, style='solid')

# Draw negative edges (dashed lines)
nx.draw_networkx_edges(G_signed, pos, edgelist=negative_edges, 
                        edge_color='red', width=2, style='dashed')

# Draw labels
nx.draw_networkx_labels(G_signed, pos, font_size=12, font_weight='bold')

plt.title('Signed Network (Green=Positive, Red=Negative)')
plt.show()

## 2. Balance Theory Analysis

Analyze structural balance in signed networks.

In [ ]:
def check_balance(G):
    """Check structural balance of triangles"""
    triangles = list(nx.enumerate_all_cliques(G))
    triangles = [t for t in triangles if len(t) == 3]
    
    balanced_triangles = 0
    total_triangles = len(triangles)
    
    for triangle in triangles:
        signs = []
        for i in range(3):
            for j in range(i+1, 3):
                if G.has_edge(triangle[i], triangle[j]):
                    signs.append(G[triangle[i]][triangle[j]]['sign'])
        
        # Check if triangle is balanced (product of signs is positive)
        if len(signs) == 3 and np.prod(signs) > 0:
            balanced_triangles += 1
    
    return balanced_triangles, total_triangles

# Analyze balance
balanced, total = check_balance(G_signed)
balance_ratio = balanced / total if total > 0 else 0

print(f"Balance Analysis:")
print(f"Balanced triangles: {balanced}")
print(f"Total triangles: {total}")
print(f"Balance ratio: {balance_ratio:.3f}")

if balance_ratio > 0.5:
    print("The network is mostly balanced!")
else:
    print("The network has significant imbalance.")

## 3. Signed Centrality Measures

Calculate centrality measures adapted for signed networks.

In [ ]:
def signed_degree_centrality(G):
    """Calculate signed degree centrality"""
    centrality = {}
    for node in G.nodes():
        positive_degree = sum(1 for neighbor in G.neighbors(node) 
                            if G[node][neighbor]['sign'] == 1)
        negative_degree = sum(1 for neighbor in G.neighbors(node) 
                            if G[node][neighbor]['sign'] == -1)
        centrality[node] = positive_degree - negative_degree
    return centrality

# Calculate signed centrality
signed_centrality = signed_degree_centrality(G_signed)

print("Signed Degree Centrality:")
for node, centrality in signed_centrality.items():
    print(f"Node {node}: {centrality}")

# Visualize with centrality coloring
plt.figure(figsize=(10, 8))

# Color nodes by signed centrality
node_colors = [signed_centrality[node] for node in G_signed.nodes()]

nx.draw_networkx_nodes(G_signed, pos, node_color=node_colors, 
                        cmap=plt.cm.RdYlBu, node_size=500)

# Draw edges
nx.draw_networkx_edges(G_signed, pos, edgelist=positive_edges, 
                        edge_color='green', width=2, style='solid')
nx.draw_networkx_edges(G_signed, pos, edgelist=negative_edges, 
                        edge_color='red', width=2, style='dashed')

nx.draw_networkx_labels(G_signed, pos, font_size=12, font_weight='bold')

plt.title('Signed Network with Signed Degree Centrality')
plt.colorbar(plt.cm.ScalarMappable(cmap=plt.cm.RdYlBu), label='Signed Centrality')
plt.show()